<a href="https://colab.research.google.com/github/kiru883/Kaggle-IEEE-CIS-Fraud-Detection/blob/master/feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive, files
drive.mount('/content/gdrive')

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import TimeSeriesSplit, train_test_split

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Load datasets

In [0]:
#####LOAD DATASETS
# train
data_trainTR = pd.read_csv("/content/gdrive/My Drive/frauds_datasets/train_transaction.csv")
data_trainID = pd.read_csv("/content/gdrive/My Drive/frauds_datasets/train_identity.csv")
data_trainID = pd.concat([data_trainTR['TransactionID'].rename(data_trainTR['TransactionID'], axis='index'),
                          data_trainID.rename(data_trainID['TransactionID'], axis='index')], axis=1).reset_index(drop=True)
# test
data_testTR = pd.read_csv("/content/gdrive/My Drive/frauds_datasets/test_transaction.csv")
data_testID = pd.read_csv("/content/gdrive/My Drive/frauds_datasets/test_identity.csv")
data_testID = pd.concat([data_testTR['TransactionID'].rename(data_testTR['TransactionID'], axis='index'),
                          data_testID.rename(data_testID['TransactionID'], axis='index')], axis=1).reset_index(drop=True)     

# Memory usage reduction function

In [0]:
def reduce_mem_usage(df):
    mem_usage_before = np.around(df.memory_usage().sum() / 1028**2)
    print(f"Memory usage before: {mem_usage_before} MB")

    for column in df.columns:
        integer_float = df[column].dropna().apply(float.is_integer).all()

        if df[column].dtype == 'float':
            mn, mx = df[column].min(), df[column].max()
            if mn > -2147483648 or mx < 2147483648:
                df[column] = df[column].astype('float32')

        elif df[column].dtype == 'int':
            mn, mx = df[column].min(), df[column].max()
            if mn > -128 or mx < 127:
                df[column] = df[column].astype('int8')
            elif mn > -32000 or mx < 32000:
                df[column] = df[column].astype('int16')
            elif mn > -2147483648 or mx < 2147483648:
                df[column] = df[column].astype('int32')

        elif df[column].dtype == 'object':
            df[column] = df[column].astype('category')

    mem_usage_after = np.around(df.memory_usage().sum() / 1028**2)
    print(f"Memory usage after: {mem_usage_after} MB")
    print(f"Optimization: {np.around(100*(1 - mem_usage_after/mem_usage_before))}%")

    return df

# Main pipelines

In [0]:
#all my hypotises
def pipeline(df):
    

Memory usage before: 1761.0 MB
Memory usage after: 851.0 MB
Optimization: 52.0%
